# **Problem Statement**

In today’s volatile financial markets, investors face significant challenges in balancing the trade-off between maximizing returns and managing risk. Market uncertainty, economic fluctuations, and geopolitical events can all significantly impact asset performance, making portfolio management more complex. As investors seek to grow their wealth, they must also mitigate potential risks that could erode their returns. Diversification, while a widely accepted strategy to reduce risk, requires a thoughtful and data-driven approach. Allocating capital effectively across asset classes such as stocks, bonds, and sectors like technology, healthcare, and financials demands a nuanced understanding of risk-return trade-offs. Investors must navigate these decisions while considering both the inherent volatility of each asset and how assets correlate with one another.

This project aims to develop an optimization model that helps investors allocate their assets in a way that maximizes risk-adjusted returns. The model will consider critical factors such as budget, sector diversification, asset diversification, and individual asset risk. By leveraging historical data from sources such as Yahoo Finance and Alpha Vantage, the model will generate optimal asset weights for a portfolio that meets predefined return goals while minimizing risk exposure. With constraints on sector and asset class exposure, the model ensures that the portfolio is well-diversified, reducing the potential for concentrated losses. This approach provides investors with a systematic way to optimize their portfolios, balancing returns and risk more effectively in unpredictable market conditions.


# **Dataset**

Data Source: Yahoo Finance (2014 - 2023)
- Assets & Time Period: This dataset includes 20 assets over a 10-year period (2014–2023). The assets consist of 4 stocks each from the Finance, Healthcare, Technology, and Semiconductor sectors (16 stocks in total), along with U.S. Treasury bonds, Gold, the S&P 500 index, and the NASDAQ 100 index.

- Yearly Data: The dataset provides yearly returns, beta (market risk), and volatility for each asset, resulting in 10 values for each metric per asset over the 10-year period.

- Key Metrics:
  1. Return: The annual average yield of each asset, calculated based on historical prices.
  2. Beta (Market Risk): A measure of each asset's volatility relative to the overall market (using the S&P 500 as the benchmark).
    - Beta > 1: Indicates the asset is more volatile than the market.
    - Beta < 1: Indicates the asset is less volatile than the market.
    - Beta = 1: Indicates the asset moves in tandem with the market.
  3. Volatility: Yearly historical volatility, which reflects the price fluctuation risk of each asset.


> **Historical Price Data**




In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the list of iconic assets
tickers = [
    "JPM", "BAC", "C", "GS",          # Finance: JPMorgan, BOA, Citigroup, Goldman Sachs
    "JNJ", "PFE", "UNH", "MRK",       # Healthcare: Johnson & Johnson, Pfizer, UnitedHealth Group, Merck & Co.
    "AAPL", "MSFT", "GOOGL", "META",  # Technology: Apple, Microsoft, Google, Meta
    "NVDA", "AMD", "INTC", "TSM",     # Semiconductor: NVIDIA, AMD, Intel, TSMC
    "TLT", "GLD", "SPY", "QQQ"        # 20+ Year Treasury Bond, Gold, S&P 500, NAS 100 (non-financial)
]

# Define the date range
start_date = "2014-01-01"
end_date = "2023-12-31"

# Fetch data
data = yf.download(tickers, start=start_date, end=end_date)
adj_close_data = data['Adj Close']

# Fill missing data
adj_close_data = adj_close_data.fillna(method='ffill').fillna(method='bfill')

# Get unique years
years = pd.to_datetime(adj_close_data.index).year.unique()
returns_list = []
betas_list = []
volatility_list = []

# Calculate returns, beta, volatility for each year
for year in years:
    yearly_data = adj_close_data.loc[f"{year}-01-01":f"{year}-12-31"]

    ### Returns
    returns = (yearly_data.iloc[-1] - yearly_data.iloc[0]) / yearly_data.iloc[0]
    # Save result to list
    for ticker, value in returns.items():
        returns_list.append({"date": year, "ticker": ticker, "value": value})

    ### Beta
    daily_returns = yearly_data.pct_change().dropna()

    # Define the benchmark
    benchmark = daily_returns['SPY']

    # Calculate Beta for each asset compare to SPY
    for ticker in tickers:
        covariance = daily_returns[ticker].cov(benchmark)
        variance = benchmark.var()
        beta = covariance / variance
        betas_list.append({"date": year, "ticker": ticker, "value": beta})

    ### Volatility
    daily_volatility = daily_returns.std()

    # Annualize the volatility for the year
    trading_days = 252  # Typical number of trading days in a year
    annual_volatility = daily_volatility * np.sqrt(trading_days)

    # Save volatility to list
    for ticker, value in annual_volatility.items():
        volatility_list.append({"date": year, "ticker": ticker, "value": value})

# Convert results to DataFrames
returns_df = pd.DataFrame(returns_list)
betas_df = pd.DataFrame(betas_list)
betas_df = betas_df.sort_values(by=['date', 'ticker']).reset_index(drop=True)
volatility_df = pd.DataFrame(volatility_list)

[*********************100%***********************]  20 of 20 completed
<ipython-input-50-f3741ee7bab9>:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  adj_close_data = adj_close_data.fillna(method='ffill').fillna(method='bfill')


> **Return**

In [ ]:
returns_df

,date,ticker,value
0,2014,AAPL,0.426283
1,2014,AMD,-0.324051
2,2014,BAC,0.119253
3,2014,C,0.036037
...,...,...,...
196,2023,SPY,0.267092
197,2023,TLT,0.008434
198,2023,TSM,0.432141
199,2023,UNH,0.030380


> **Beta (Market Risk)**


In [ ]:
betas_df

,date,ticker,value
0,2014,AAPL,0.846730
1,2014,AMD,1.214122
2,2014,BAC,1.201042
3,2014,C,1.271266
...,...,...,...
196,2023,SPY,1.000000
197,2023,TLT,0.183284
198,2023,TSM,1.298732
199,2023,UNH,0.279227


> **Historical Volatility**

In [ ]:
volatility_df

,date,ticker,value
0,2014,AAPL,0.216445
1,2014,AMD,0.406660
2,2014,BAC,0.214164
3,2014,C,0.204732
...,...,...,...
196,2023,SPY,0.130915
197,2023,TLT,0.183684
198,2023,TSM,0.308128
199,2023,UNH,0.211068


# **Model**

## **Model1: Maximize the profit**



> **Objective**

The objective of this model is to maximize the expected return of the portfolio.


> **Decision Variables**

The weight (allocation percentage) of each asset in the portfolio.


> **Constraints**


1. Budget Constraint:
  - The total allocation of assets must equal 100% of the investment.
  - No short selling is allowed.

2. Asset Diversification:
  - A minimum of 5 assets must be included in the portfolio, selected from the 20 available.
  - Allocation to individual assets cannot exceed 15% of the portfolio, ensuring no single asset is overly concentrated.
  - The portfolio must include a balance between different asset classes, such as stocks and bonds, to maintain asset diversification.
    -  50% allocated to stocks, 10% to bonds, 5% to commodities, 5% to ETFs

3. Sector Diversification:
  - Allocation to any single sector cannot exceed 30% of the total portfolio.
  - The portfolio must include at least one company from the technology sector.

4. Risk Constraints:
  - This constraint ensures the portfolio's total beta, a measure of volatility relative to the market, stays below 1.5 by limiting the weighted sum of individual asset betas.




In [ ]:
import cvxpy as cp
import pandas as pd
import numpy as np

# Get unique years and tickers
years = returns_df['date'].unique()
tickers = returns_df['ticker'].unique()

# Define containers for results
results = []

# Asset Class Definitions
stocks = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]  # Indices for stocks
bonds = [16]  # Indices for bonds (e.g., "TLT")
commodities = [17]  # Indices for commodities (e.g., "GLD")
etfs = [18, 19]  # Indices for ETFs (e.g., "SPY", "QQQ")

# Sector Definitions
finance_sector = [0, 1, 2, 3]
healthcare_sector = [4, 5, 6, 7]
tech_sector = [8, 9, 10, 11]
semi_sector = [12, 13, 14, 15]

# Run the optimization model for each year
for year in years:
    # Filter data for the current year
    year_returns = returns_df[returns_df['date'] == year].set_index('ticker')['value'].values
    year_betas = betas_df[betas_df['date'] == year].set_index('ticker')['value'].values
    year_volatility = volatility_df[volatility_df['date'] == year].set_index('ticker')['value'].values

    # Covariance matrix approximation (diagonal matrix from volatilities)
    cov_matrix = np.diag(year_volatility**2)

    ### Decision Variables
    n_assets = len(tickers)
    w = cp.Variable(n_assets)
    z = cp.Variable(n_assets)  # Binary variable indicating selection of asset

    ### Objective: Maximize expected return
    objective = cp.Maximize(year_returns @ w)

    ### Constraints
    constraints = [
        ##### Budget Constraints
        cp.abs(cp.sum(w) - 1) <= 1e-5,  # The total allocation of assets must equal 100% of the investment.
        w >= 0,          # No short selling is allowed.
        z >= 0,          # z is non-negative
        z <= 1,          # z is at most 1

        ##### Asset Constraints
        cp.sum(z) >= 5,  # At least 5 assets in the portfolio
        w <= 0.15 * z,   # No single asset exceeds 15% of the portfolio
        cp.sum(w[stocks]) >= 0.5, # At least 50% of the portfolio must be allocated to stocks
        cp.sum(w[bonds]) >= 0.1,  # At least 10% of the portfolio must be allocated to bonds
        cp.sum(w[commodities]) >= 0.05, # At least 5% allocated to commodities
        cp.sum(w[etfs]) >= 0.05,  # At least 5% allocated to ETFs
    ]

    ### Sector Constraints
    # Allocation to any single sector cannot exceed 30% of the total portfolio.
    constraints.append(cp.sum(w[finance_sector]) <= 0.3)
    constraints.append(cp.sum(w[healthcare_sector]) <= 0.3)
    constraints.append(cp.sum(w[tech_sector]) <= 0.3)
    constraints.append(cp.sum(w[semi_sector]) <= 0.3)
    # The portfolio must include at least one company from the technology sector.
    constraints.append(cp.sum(w[tech_sector]) >= 0.0001)

    ##### Risk Constraints
    # Portfolio beta constraint (beta < 1.5)
    constraints.append(cp.sum(cp.multiply(year_betas, w)) <= 1.5)

    # Problem setup and solve
    problem = cp.Problem(objective, constraints)
    result_status = problem.solve(solver=cp.GLPK_MI)

    # Check if the solution is feasible
    if problem.status not in ["infeasible", "unbounded"]:
        # Store results for the current year
        optimal_weights = w.value
        portfolio_return = np.dot(optimal_weights, year_returns)
        portfolio_variance = np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))
        portfolio_std_dev = np.sqrt(portfolio_variance)

        results.append({
            "Year": year,
            "Optimal Weights": dict(zip(tickers, optimal_weights)),
            "Portfolio Return": portfolio_return,
            "Portfolio Risk": portfolio_std_dev
        })
    else:
        print(f"Optimization failed for year {year} with status: {problem.status}")

> **Weight**


In [ ]:
# Weights Value
weights_records = []
for result in results:
    year = result["Year"]
    for ticker, weight in result["Optimal Weights"].items():
        weights_records.append({"Year": year, "Ticker": ticker, "Weight": weight})
weights_df = pd.DataFrame(weights_records)

# Reshape the DataFrame using pivot
weights_pivot_df = weights_df.pivot(index='Year', columns='Ticker', values='Weight')

# Reset the display option for better visualization if needed
pd.set_option('display.max_rows', len(weights_pivot_df))  # Adjust max rows to a reasonable number

# Display the pivoted DataFrame
display(weights_pivot_df)

Ticker,AAPL,AMD,BAC,C,GLD,GOOGL,GS,INTC,JNJ,JPM,META,MRK,MSFT,NVDA,PFE,QQQ,SPY,TLT,TSM,UNH
Year,,,,,,,,,,,,,,,,,,,,
2014,0.15,-0.00,-0.00000,-0.0,-0.00,-0.00000,-0.00000,0.15000,-0.00,-0.00000,0.150000,-0.00,-0.00,0.100010,-0.00,-0.00,0.1,0.05,0.15,0.15000
2015,-0.00,-0.00,-0.00000,-0.0,-0.00,0.15000,-0.00000,-0.00000,-0.00,0.10001,0.150000,-0.00,0.15,0.150000,-0.00,-0.00,0.1,0.05,-0.00,0.15000
2016,-0.00,0.15,0.10001,-0.0,-0.00,-0.00000,0.15000,-0.00000,-0.00,0.15000,-0.000000,-0.00,-0.00,0.150000,-0.00,-0.00,0.1,0.05,-0.00,0.15000
2017,0.15,-0.00,-0.00000,-0.0,-0.00,-0.00000,-0.00000,-0.00000,-0.00,-0.00000,0.150000,-0.00,0.15,0.150000,-0.00,-0.00,0.1,0.05,0.15,0.10001
2018,-0.00,0.15,-0.00000,-0.0,-0.00,-0.00000,-0.00000,0.10001,-0.00,-0.00000,-0.000000,0.15,0.15,-0.000000,0.15,-0.00,0.1,0.05,-0.00,0.15000
2019,0.15,0.15,-0.00000,-0.0,-0.00,-0.00000,-0.00000,-0.00000,-0.00,-0.00000,0.140834,-0.00,0.15,0.109176,-0.00,-0.00,0.1,0.05,0.15,-0.00000
2020,0.15,0.15,-0.00000,-0.0,-0.00,-0.00000,-0.00000,-0.00000,-0.00,-0.00000,0.100010,-0.00,-0.00,0.150000,-0.00,0.15,0.1,0.05,0.15,-0.00000
2021,-0.00,0.15,0.15000,-0.0,-0.00,0.15000,0.04991,-0.00000,-0.00,0.00010,-0.000000,-0.00,-0.00,0.150000,0.15,-0.00,0.1,0.05,-0.00,0.05000
2022,-0.00,-0.00,-0.00000,-0.0,0.15,-0.00000,0.09999,-0.00000,0.15,-0.00000,-0.000000,0.15,-0.00,-0.000000,0.15,-0.00,0.1,0.05,-0.00,0.15000


> **Result**


In [ ]:
# Final Result
Result = pd.DataFrame({
    "Year": [result["Year"] for result in results],
    "Expected Return": [result["Portfolio Return"] for result in results],
    "Risk": [result["Portfolio Risk"] for result in results]
})
Result

,Year,Expected Return,Risk
0,2014,0.359154,0.088205
1,2015,0.288630,0.100519
2,2016,1.037572,0.162111
3,2017,0.467103,0.079855
4,2018,0.247148,0.123095
5,2019,0.725934,0.118441
6,2020,0.679054,0.177963
7,2021,0.623415,0.116020
8,2022,0.039412,0.081301
9,2023,1.143377,0.138891


**The top-performing years—2023 (114.34%), 2016 (103.76%), and 2019 (72.59%)—showcase the importance of strategic allocations across key sectors. In 2023, the technology sector, including companies like Microsoft (MSFT), Meta (META), and Google (GOOGL), along with semiconductors such as NVIDIA (NVDA) and Taiwan Semiconductor (TSM), delivered outstanding returns due to strong post-pandemic recovery and demand for AI-driven solutions. Similarly, 2016 saw gains from semiconductors (NVDA, AMD) and finance (Goldman Sachs, GS), fueled by growing tech adoption and economic stabilization, albeit with a slightly higher risk (16.21%). In 2019, technology (AAPL, MSFT, META) and semiconductor stocks (TSM) excelled, supported by global economic growth and digital transformation trends, with a balanced risk-return profile (11.84%). These years highlight the dominance of technology and semiconductor sectors, complemented by contributions from finance and healthcare, in driving exceptional portfolio performance.**

## **Model2: Minimize the Risk**



> **Objective**

The objective of this model is to maximize the volatilities of the portfolio.


> **Decision Variables**

Same as Model 1

> **Constraints**


1. Budget Constraint:
  - The total allocation of assets must equal 100% of the investment.
  - No short selling is allowed.

2. Asset Diversification:
  - A minimum of 5 assets must be included in the portfolio, selected from the 20 available.
  - Allocation to individual assets cannot exceed 15% of the portfolio, ensuring no single asset is overly concentrated.
  - The portfolio must include a balance between different asset classes, such as stocks and bonds, to maintain asset diversification.
    -  50% allocated to stocks, 10% to bonds, 5% to commodities, 5% to ETFs

3. Sector Diversification:
  - Allocation to any single sector cannot exceed 30% of the total portfolio.
  - The portfolio must include at least one company from the technology sector.

4. Return Constraints:
  - This constraint ensures the portfolio's total expected return exceeds 4% by requiring the weighted sum of individual asset returns to be at least 0.04.




In [ ]:
import cvxpy as cp
import pandas as pd
import numpy as np

# Get unique years and tickers
years = returns_df['date'].unique()
tickers = returns_df['ticker'].unique()

# Define containers for results
results = []
stocks = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]  # Indices for stocks
bonds = [16]  # Indices for bonds (e.g., "TLT")
commodities = [17]  # Indices for commodities (e.g., "GLD")
etfs = [18, 19]  # Indices for ETFs (e.g., "SPY", "QQQ")

# Sector Definitions
finance_sector = [0, 1, 2, 3]
healthcare_sector = [4, 5, 6, 7]
tech_sector = [8, 9, 10, 11]
semi_sector = [12, 13, 14, 15]

# Run the optimization model for each year
for year in years:
    # Filter data for the current year
    year_returns = returns_df[returns_df['date'] == year].set_index('ticker')['value'].values
    year_betas = betas_df[betas_df['date'] == year].set_index('ticker')['value'].values
    year_volatility = volatility_df[volatility_df['date'] == year].set_index('ticker')['value'].values

    # Covariance matrix approximation (diagonal matrix from volatilities)
    cov_matrix = np.diag(year_volatility**2)

    ### Decision Variables
    n_assets = len(tickers)
    w = cp.Variable(n_assets)
    z = cp.Variable(n_assets)  # Binary variable indicating selection of asset

    ### Objective: Minimize volatilities
    objective = cp.Minimize(w.T @ cov_matrix @ w)

    ### Constraints
    constraints = [
        ##### Budget Constraints
        cp.abs(cp.sum(w) - 1) <= 1e-5,  # The total allocation of assets must equal 100% of the investment.
        w >= 0,          # No short selling
        w <= 0.15 * z,   # Asset weight is limited by selection variable
        z >= 0,          # z is non-negative
        z <= 1,          # z is at most 1
        cp.sum(z) >= 5,  # At least 5 assets are selected
        cp.sum(w[stocks]) >= 0.5,  # At least 50% in stocks
        cp.sum(w[bonds]) >= 0.1,   # At least 10% in bonds
        cp.sum(w[commodities]) >= 0.05,  # At least 5% in commodities
        cp.sum(w[etfs]) >= 0.05,   # At least 5% in ETFs
        cp.sum(w[finance_sector]) <= 0.3,  # Sector constraints
        cp.sum(w[healthcare_sector]) <= 0.3,
        cp.sum(w[tech_sector]) <= 0.3,
        cp.sum(w[semi_sector]) <= 0.3,
        cp.sum(w[tech_sector]) >= 0.0001,  # Must include at least one tech company
        #expected_returns @ w >= 0.04,  # Minimum portfolio expected return of 4%
]

    ##### Return Constraints
    # Portfolio return constraint (return > 0.04)
    constraints.append(cp.sum(cp.multiply(year_returns, w)) >= 0.04)

    # Problem setup and solve
    problem = cp.Problem(objective, constraints)
    result_status = problem.solve()

    # Check if the solution is feasible
    if problem.status not in ["infeasible", "unbounded"]:
        # Store results for the current year
        optimal_weights = w.value
        portfolio_return = np.dot(optimal_weights, year_returns)
        portfolio_variance = np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))
        portfolio_std_dev = np.sqrt(portfolio_variance)

        results.append({
            "Year": year,
            "Optimal Weights": dict(zip(tickers, optimal_weights)),
            "Portfolio Return": portfolio_return,
            "Portfolio Risk": portfolio_std_dev
        })
    else:
        print(f"Optimization failed for year {year} with status: {problem.status}")

Optimization failed for year 2022 with status: infeasible


> **Weight**


In [ ]:
# Weights Value
weights_records = []
for result in results:
    year = result["Year"]
    for ticker, weight in result["Optimal Weights"].items():
        weights_records.append({"Year": year, "Ticker": ticker, "Weight": weight})
weights_df = pd.DataFrame(weights_records)

# Reshape the DataFrame using pivot
weights_pivot_df = weights_df.pivot(index='Year', columns='Ticker', values='Weight')

# Reset the display option for better visualization if needed
pd.set_option('display.max_rows', len(weights_pivot_df))  # Adjust max rows to a reasonable number

# Display the pivoted DataFrame
display(weights_pivot_df)

Ticker,AAPL,AMD,BAC,C,GLD,GOOGL,GS,INTC,JNJ,JPM,META,MRK,MSFT,NVDA,PFE,QQQ,SPY,TLT,TSM,UNH
Year,,,,,,,,,,,,,,,,,,,,
2014,0.032488,0.009203,0.033184,0.036312,0.068778,0.033500,0.050615,0.030814,0.071629,0.047642,0.011901,0.041637,0.041987,0.023833,0.058624,0.079592,0.120869,0.134524,0.029800,0.043070
2015,0.032163,0.007441,0.034834,0.038994,0.115926,0.027582,0.046216,0.038469,0.085705,0.046474,0.034837,0.048095,0.028875,0.019298,0.057333,0.071909,0.099966,0.094690,0.035509,0.035686
2016,0.036568,0.002766,0.019342,0.019741,0.076642,0.049917,0.027770,0.039561,0.112630,0.033295,0.024934,0.045293,0.038808,0.009028,0.057098,0.077503,0.118215,0.119557,0.035858,0.055472
2017,0.030307,0.002772,0.020495,0.030725,0.096374,0.040804,0.024266,0.031988,0.071862,0.035755,0.032685,0.038752,0.042933,0.005839,0.075978,0.088783,0.149994,0.099580,0.033896,0.046213
2018,0.025358,0.012227,0.028974,0.022805,0.149979,0.028380,0.016992,0.021187,0.045222,0.042418,0.009282,0.095416,0.038511,0.004778,0.082367,0.043678,0.099937,0.149984,0.028982,0.053530
2019,0.026636,0.006282,0.034003,0.030876,0.133920,0.032434,0.033011,0.024883,0.067524,0.052290,0.023753,0.054007,0.046331,0.011059,0.050487,0.069436,0.116263,0.127983,0.030080,0.028750
2020,0.031398,0.018489,0.019869,0.015052,0.149991,0.046235,0.025042,0.023963,0.074443,0.023102,0.032440,0.067342,0.035466,0.020416,0.053445,0.053810,0.099931,0.147057,0.032952,0.029561
2021,0.034085,0.011747,0.032611,0.032497,0.116635,0.036103,0.033441,0.021486,0.101280,0.046655,0.024380,0.038525,0.048537,0.010497,0.031242,0.064459,0.127562,0.110548,0.019667,0.058041
2023,0.057782,0.010442,0.031805,0.035394,0.128206,0.024898,0.039527,0.015514,0.083980,0.053297,0.014503,0.064692,0.036460,0.009798,0.043434,0.071959,0.134241,0.068190,0.024233,0.051644


> **Result**

In [ ]:
# Final Result
Result = pd.DataFrame({
    "Year": [result["Year"] for result in results],
    "Expected Return": [result["Portfolio Return"] for result in results],
    "Risk": [result["Portfolio Risk"] for result in results]
})
Result

,Year,Expected Return,Risk
0,2014,0.179950,0.039013
1,2015,0.044002,0.048104
2,2016,0.177598,0.044719
3,2017,0.237222,0.030055
4,2018,0.039939,0.047846
5,2019,0.315191,0.042744
6,2020,0.236009,0.084317
7,2021,0.246663,0.046184
8,2023,0.253348,0.047966


**In the Model 2 results, the portfolio includes all assets, as this strategy typically results in a more diversified portfolio, spreading investments across various assets to reduce overall volatility. Gold consistently has higher weights compared to individual stocks, with values often exceeding 10% in most years. This indicates that the portfolio optimization process views GLD as a stabilizing or low-risk component. JNJ also has the relative higher weights as Johnson & Johnson is considered a defensive stock that it tends to perform relatively well during economic downturns due to its essential healthcare products and services.Some tech stocks (e.g., MSFT, META, AMD) have relatively lower weights, suggesting a more balanced approach rather than heavily favoring tech-focused allocations. In 2020, the risk is relatively high due to the greater allocations to volatile assets (e.g., tech stocks) and reduced exposure to stabilizing assets like GLD and TLT.**

# **Sensitivity analysis**

## Model1

In [ ]:
def round_and_normalize(weights, decimals=4):
    rounded_weights = np.round(weights, decimals)
    total = np.sum(rounded_weights)
    if total != 1:
        # Adjust weights proportionally to ensure they sum to 1
        adjustment_factor = 1 / total
        rounded_weights = np.round(rounded_weights * adjustment_factor, decimals)
    return rounded_weights

def round_shadow_prices(shadow_prices, constraint_labels, decimals=4):
    rounded_prices = {}
    for label, value in zip(constraint_labels, shadow_prices.values()):
        if isinstance(value, np.ndarray):
            rounded_prices[label] = np.round(value, decimals).tolist()  # Vectorized constraints
        else:
            rounded_prices[label] = round(value, decimals)  # Scalar constraints
    return rounded_prices

# Define constraint labels for clarity
constraint_labels = [
    "Budget Constraint",
    "No Short Selling Constraint",
    "z greater equal than 0",
    "z smaller equal than 1",
    "Minimum Asset Count Constraint",
    "Max 15% Per Asset Constraint",
    "At Least 50% in Stocks",
    "At Least 10% in Bonds",
    "At Least 5% in Commodities",
    "At Least 5% in ETFs",
    "Finance Sector Max 30%",
    "Healthcare Sector Max 30%",
    "Tech Sector Max 30%",
    "Semiconductor Sector Max 30%",
    "At Least One Tech Company",
    "Portfolio Beta Constraint",
]

# Run the optimization model for each year
results = []
sensitivity_analysis = []

for year in years:
    # Filter data for the current year
    year_returns = returns_df[returns_df['date'] == year].set_index('ticker')['value'].values
    year_betas = betas_df[betas_df['date'] == year].set_index('ticker')['value'].values
    year_volatility = volatility_df[volatility_df['date'] == year].set_index('ticker')['value'].values

    # Covariance matrix approximation (diagonal matrix from volatilities)
    cov_matrix = np.diag(year_volatility**2)

    ### Decision Variables
    n_assets = len(tickers)
    w = cp.Variable(n_assets)
    z = cp.Variable(n_assets)

    ### Objective: Maximize expected return
    objective = cp.Maximize(year_returns @ w)

    ### Constraints
    constraints = [
        cp.abs(cp.sum(w) - 1) <= 1e-5,  # Budget constraint
        w >= 0,  # No short selling
        z >= 0,  # z is non-negative
        z <= 1,  # z is at most 1
        cp.sum(z) >= 5,  # Minimum asset count
        w <= 0.15 * z,  # Max 15% per asset
        cp.sum(w[stocks]) >= 0.5,  # At least 50% in stocks
        cp.sum(w[bonds]) >= 0.1,  # At least 10% in bonds
        cp.sum(w[commodities]) >= 0.05,  # At least 5% in commodities
        cp.sum(w[etfs]) >= 0.05,  # At least 5% in ETFs
        cp.sum(w[finance_sector]) <= 0.3,  # Finance sector max 30%
        cp.sum(w[healthcare_sector]) <= 0.3,  # Healthcare sector max 30%
        cp.sum(w[tech_sector]) <= 0.3,  # Tech sector max 30%
        cp.sum(w[semi_sector]) <= 0.3,  # Semiconductor sector max 30%
        cp.sum(w[tech_sector]) >= 0.0001,  # At least one tech company
    ]

    constraints.append(cp.sum(cp.multiply(year_betas, w)) <= 1.5)  # Portfolio beta constraint

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    result_status = problem.solve(solver=cp.ECOS)

    if problem.status not in ["infeasible", "unbounded"]:
        optimal_weights = w.value

        # Correct small numerical negatives
        optimal_weights = np.maximum(optimal_weights, 0)

        # Normalize and round weights
        optimal_weights = round_and_normalize(optimal_weights, decimals=4)

        portfolio_return = np.dot(optimal_weights, year_returns)
        portfolio_variance = np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))
        portfolio_std_dev = np.sqrt(portfolio_variance)

        results.append({
            "Year": year,
            "Portfolio Return": portfolio_return,
            "Portfolio Risk": portfolio_std_dev,
            "Optimal Weights": dict(zip(tickers, optimal_weights))
        })

        # Sensitivity Analysis: Get shadow prices
        shadow_prices = {label: c.dual_value for label, c in zip(constraint_labels, problem.constraints)}
        rounded_shadow_prices = round_shadow_prices(shadow_prices, constraint_labels, decimals=4)
        sensitivity_analysis.append({
            "Year": year,
            **rounded_shadow_prices
        })
    else:
        print(f"Optimization failed for year {year} with status: {problem.status}")

# Convert sensitivity analysis to a DataFrame
sensitivity_df = pd.DataFrame(sensitivity_analysis)
#results_df = pd.DataFrame(results)


/usr/local/lib/python3.10/dist-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


In [ ]:
sensitivity_df

,Year,Budget Constraint,No Short Selling Constraint,z greater equal than 0,z smaller equal than 1,Minimum Asset Count Constraint,Max 15% Per Asset Constraint,At Least 50% in Stocks,At Least 10% in Bonds,At Least 5% in Commodities,At Least 5% in ETFs,Finance Sector Max 30%,Healthcare Sector Max 30%,Tech Sector Max 30%,Semiconductor Sector Max 30%,At Least One Tech Company,Portfolio Beta Constraint
0,2014,0.2868,"[0.0, 0.6109, 0.1676, 0.2508, 0.3243, 0.3343, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0209, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0247,...",0.0,"[0.1395, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1649,...",0.0,0.1412,0.0177,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0
1,2015,0.0783,"[0.0992, 0.0034, 0.1266, 0.1218, 0.189, 0.0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0586, 0.0, 0.0, 0....",0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.3908, 0.0, 0.0, 0....",0.0,0.0655,0.1070,0.0000,0.0000,0.0,0.0000,0.0917,0.0000,0.0
2,2016,0.3662,"[0.2423, 0.0, 0.0, 0.1931, 0.3008, 0.3227, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.4092, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0, ...",0.0,"[0.0, 2.7277, 0.0, 0.0, 0.0, 0.0, 0.0066, 0.0,...",0.0,0.2303,0.3617,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0
3,2017,0.3861,"[0.0, 0.4868, 0.0552, 0.1402, 0.2668, 0.0824, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0141, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",0.0,"[0.0943, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",0.0,0.1783,0.2990,0.0000,0.0000,0.0,0.0000,0.0047,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,2020,0.3021,"[0.0, 0.0, 0.584, 0.6675, 0.0631, 0.0216, 0.14...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0488, 0.0616, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.0,"[0.3251, 0.4106, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.0,0.1298,0.1338,0.0000,0.1551,0.0,0.0000,0.1247,0.0000,0.0
7,2021,0.4689,"[0.1002, 0.0, 0.0, 0.446, 0.5313, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0117, 0.0044, 0.0, 0.0, 0.0314, 0.0, 0...",0.0,"[0.0, 0.0782, 0.0292, 0.0, 0.0, 0.2094, 0.0, 0...",0.0,0.1639,0.5138,0.0120,0.0119,0.0,0.0000,0.1373,0.1793,0.0
8,2022,0.1085,"[0.1735, 0.4604, 0.1576, 0.1459, 0.0, 0.283, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0174, 0.0, 0.0, 0.0, 0....",0.0,"[0.0, 0.0, 0.0, 0.0, 0.1163, 0.0, 0.0, 0.0, 0....",0.0,0.0780,0.1853,0.0000,0.0000,0.0,0.0776,0.0000,0.0000,0.0
9,2023,0.5674,"[0.0195, 0.0, 0.5314, 0.3919, 0.4499, 0.0, 0.4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.1103, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0535,...",0.0,"[0.0, 0.7351, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3564,...",0.0,0.3003,0.5590,0.1353,0.0000,0.0,0.0000,0.0073,0.0000,0.0


## Model2

In [ ]:
def round_and_normalize(weights, decimals=4):
    rounded_weights = np.round(weights, decimals)
    total = np.sum(rounded_weights)
    if total != 1:
        # Adjust weights proportionally to ensure they sum to 1
        adjustment_factor = 1 / total
        rounded_weights = np.round(rounded_weights * adjustment_factor, decimals)
    return rounded_weights

def round_shadow_prices(shadow_prices, constraint_labels, decimals=4):
    rounded_prices = {}
    for label, value in zip(constraint_labels, shadow_prices.values()):
        if isinstance(value, np.ndarray):
            rounded_prices[label] = np.round(value, decimals).tolist()  # Vectorized constraints
        else:
            rounded_prices[label] = round(value, decimals)  # Scalar constraints
    return rounded_prices

# Define constraint labels for clarity
constraint_labels = [
    "Budget Constraint",
    "No Short Selling Constraint",
    "Weight Limited by Selection Variable",
    "z greater equal than 0",
    "z smaller equal than 1",
    "Minimum Asset Count Constraint",
    "At Least 50% in Stocks",
    "At Least 10% in Bonds",
    "At Least 5% in Commodities",
    "At Least 5% in ETFs",
    "Finance Sector Max 30%",
    "Healthcare Sector Max 30%",
    "Tech Sector Max 30%",
    "Semiconductor Sector Max 30%",
    "At Least One Tech Company",
    "Portfolio Minimum Expected Return (>= 4%)",
]

# Run the optimization model for each year
results = []
sensitivity_analysis = []

for year in years:
    # Filter data for the current year
    year_returns = returns_df[returns_df['date'] == year].set_index('ticker')['value'].values
    year_betas = betas_df[betas_df['date'] == year].set_index('ticker')['value'].values
    year_volatility = volatility_df[volatility_df['date'] == year].set_index('ticker')['value'].values

    # Covariance matrix approximation (diagonal matrix from volatilities)
    cov_matrix = np.diag(year_volatility**2)

    ### Decision Variables
    n_assets = len(tickers)
    w = cp.Variable(n_assets)
    z = cp.Variable(n_assets)

    ### Objective: Minimize volatilities
    objective = cp.Minimize(w.T @ cov_matrix @ w)

    ### Constraints
    constraints = [
        ##### Budget Constraints
        cp.abs(cp.sum(w) - 1) <= 1e-5,  # The total allocation of assets must equal 100% of the investment.
        w >= 0,          # No short selling
        w <= 0.15 * z,   # Asset weight is limited by selection variable
        z >= 0,          # z is non-negative
        z <= 1,          # z is at most 1
        cp.sum(z) >= 5,  # At least 5 assets are selected
        cp.sum(w[stocks]) >= 0.5,  # At least 50% in stocks
        cp.sum(w[bonds]) >= 0.1,   # At least 10% in bonds
        cp.sum(w[commodities]) >= 0.05,  # At least 5% in commodities
        cp.sum(w[etfs]) >= 0.05,   # At least 5% in ETFs
        cp.sum(w[finance_sector]) <= 0.3,  # Sector constraints
        cp.sum(w[healthcare_sector]) <= 0.3,
        cp.sum(w[tech_sector]) <= 0.3,
        cp.sum(w[semi_sector]) <= 0.3,
        cp.sum(w[tech_sector]) >= 0.0001,  # Must include at least one tech company
        #expected_returns @ w >= 0.04,  # Minimum portfolio expected return of 4%
]

    ##### Return Constraints
    # Portfolio beta constraint (return > 0.04)
    constraints.append(cp.sum(cp.multiply(year_returns, w)) >= 0.04)

    # Problem setup and solve
    problem = cp.Problem(objective, constraints)
    result_status = problem.solve()
    # Solve the problem
    problem = cp.Problem(objective, constraints)
    result_status = problem.solve(solver=cp.ECOS)

    if problem.status not in ["infeasible", "unbounded"]:
        optimal_weights = w.value

        # Correct small numerical negatives
        optimal_weights = np.maximum(optimal_weights, 0)

        # Normalize and round weights
        optimal_weights = round_and_normalize(optimal_weights, decimals=4)

        portfolio_return = np.dot(optimal_weights, year_returns)
        portfolio_variance = np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))
        portfolio_std_dev = np.sqrt(portfolio_variance)

        results.append({
            "Year": year,
            "Portfolio Return": portfolio_return,
            "Portfolio Risk": portfolio_std_dev,
            "Optimal Weights": dict(zip(tickers, optimal_weights))
        })

        # Sensitivity Analysis: Get shadow prices
        shadow_prices = {label: c.dual_value for label, c in zip(constraint_labels, problem.constraints)}
        rounded_shadow_prices = round_shadow_prices(shadow_prices, constraint_labels, decimals=4)
        sensitivity_analysis.append({
            "Year": year,
            **rounded_shadow_prices
        })
    else:
        print(f"Optimization failed for year {year} with status: {problem.status}")

# Convert sensitivity analysis to a DataFrame
sensitivity_df = pd.DataFrame(sensitivity_analysis)
#results_df = pd.DataFrame(results)


/usr/local/lib/python3.10/dist-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


Optimization failed for year 2022 with status: infeasible


In [ ]:
sensitivity_df

,Year,Budget Constraint,No Short Selling Constraint,Weight Limited by Selection Variable,z greater equal than 0,z smaller equal than 1,Minimum Asset Count Constraint,At Least 50% in Stocks,At Least 10% in Bonds,At Least 5% in Commodities,At Least 5% in ETFs,Finance Sector Max 30%,Healthcare Sector Max 30%,Tech Sector Max 30%,Semiconductor Sector Max 30%,At Least One Tech Company,Portfolio Minimum Expected Return (>= 4%)
0,2014,0.0030,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
1,2015,0.0046,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
2,2016,0.0040,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
3,2017,0.0019,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
4,2018,0.0047,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0017, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0002, 0.0, 0.0, 0.0, 0....",0.0,0.0,0.0015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0073
5,2019,0.0037,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
6,2020,0.0137,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0023, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0, 0.0, 0....",0.0,0.0,0.0087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
7,2021,0.0043,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
8,2023,0.0046,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000


**We conducted sensitivity analyses on the two models to examine the shadow prices of various constraints. The analysis revealed that several constraints had a shadow price of 0, indicating no impact on the model's outcome. Key constraints with a shadow price of 0 included the budget constraint, the "at least 10% in bonds" constraint, and the "at least 5% in commodities" constraint in Model 1, as well as the budget constraint in Model 2. In Model 1, the budget constraint's shadow price fluctuate annually, peaking at 0.5674 in 2023, highlighting its significant influence on the model that year. In Model 2, the highest shadow price for the budget constraint was 0.0137 in 2020, suggesting that budget increases during that year could exacerbate risk. Other constraints had negligible or zero shadow prices, underscoring the budget constraint as the most critical factor for model analysis.**

# **Challenge**

- **What was the biggest challenge you faced and how you overcame it (or, tried but did not – that’s fine too – not every problem has a solution.)**

**The first challenge we faced was splitting the ten-year dataset by year. Initially, we calculated the overall portfolio return using only the first and last days of the dataset. However, this approach proved inaccurate due to the extended time span. To resolve this, we segmented the dataset into annual intervals and ran our model separately for each year. This adjustment allowed us to generate portfolio results based on data from the previous year, enhancing the reliability of our analysis.**

**Another challenge arose when applying a different model and conducting sensitivity analysis. The problem involved both continuous variables (w) and binary variables (z), categorizing it as a Mixed-Integer Linear Programming (MILP) problem. Shadow prices, however, are not directly available for MILP models since duality theory does not apply to problems with integer constraints. To address this, we converted the binary variables (z) into continuous variables by introducing additional constraints (0 ≤ z ≤ 1). This modification enabled us to conduct the analysis while preserving the model's integrity.**


# **Conclusion**

- **Model Insights:** Model 1, focused on maximizing returns, demonstrates the dominance of technology and semiconductor sectors in driving portfolio performance during top years like 2023 (114.34%), 2016 (103.76%), and 2019 (72.59%). Model 2, aimed at minimizing risk, results in a more diversified portfolio, allocating higher weights to low-risk assets like Gold and achieving balanced sector exposure.

- **Impact of Market Conditions:** The poorest performance in 2022 underscores the effects of macroeconomic challenges such as inflation and rising interest rates, which reduced business investments and led to a 19.4% decline in the S&P 500, marking its worst year since 2008.

- **Sensitivity Analysis:** Budget constraints emerge as the most critical factor influencing portfolio outcomes, especially in high-performing years like 2023. Other constraints, such as minimum allocations to bonds and commodities, showed minimal impact, highlighting the importance of budget flexibility in achieving optimal results.

# **Reference**
ChatGPT: Assisted in refining the wording and debugging the code.
